# Notebook for Airconics examples 

IMPORTANT!: ipython notebook bug requires first code cells to be run manually (i.e. not with 'Run All' Cells). After this, all cells can be rerun with 'Run all below'. 

In [1]:
import airconics
import airconics.AirCONICStools as act
%gui qt4
# Initialise the display
from OCC.Display.SimpleGui import init_display
display, start_display, add_menu, add_function_to_menu = init_display()

Using PyQt4


## Airfoils

In [ ]:
LEPoint = [0., 0., 0.] 
ChordLength = 1

AirfoilSeligName = 'dae11'

# Instantiate class to set up a generic airfoil with these basic parameters
Af1 = airconics.primitives.Airfoil(LEPoint, ChordLength=3., SeligProfile=AirfoilSeligName)
Af1_disp = display.DisplayShape(Af1.Curve, update=True);

In [ ]:
# Remove Airfoil:
display.Context.Erase(Af1_disp.GetObject().GetHandle())
Af1.Curve.GetObject().Delete()   # Not sure if this truly collects the garbage yet

# Or Clear the display:
display.EraseAll()

## Simple Lofted Wing

In [ ]:
# Add NACA 4 digit airfoils to loft between:
Af2 = airconics.primitives.Airfoil([0.,0.,0.], ChordLength=3., Naca4Profile='2412')
display.DisplayShape(Af2.Curve, update=True, color='GREEN');

Af3 = airconics.primitives.Airfoil([0., 5., 0.], ChordLength=1., Naca4Profile='0012')
display.DisplayShape(Af3.Curve, update=True, color='GREEN');

Af4 = airconics.primitives.Airfoil([0., 6., 0.2], ChordLength=0.2, Naca4Profile='0012')
display.DisplayShape(Af4.Curve, update=True, color='GREEN');

surf = act.AddSurfaceLoft([Af2, Af3, Af4])

# Note that surf is a TOPO_DS Shape, and hence no surf.Shape is required for display
display.DisplayShape(surf, update=True);

In [ ]:
display.EraseAll()

## Wing, Transonic Airliner

TODO: I tried making the lifting surface loft through airfoil sections as a solid loft, however the output stp file was not correct in Rhino (may be because TE and ends are not closed)

In [ ]:
# Import all example functional definitions for the Common Research Model (CRM) Wing:
from airconics.examples.wing_example_transonic_airliner import *

# Position of the apex of the wing
P = [0., 0., 0.]

# Class definition
NSeg = 10
ChordFactor = 1
ScaleFactor = 50

# Generate (surface building is done during construction of the class)
Wing = liftingsurface.LiftingSurface(P, mySweepAngleFunctionAirliner, 
    myDihedralFunctionAirliner, 
    myTwistFunctionAirliner, 
    myChordFunctionAirliner, 
    myAirfoilFunctionAirliner, SegmentNo=NSeg, ScaleFactor=ScaleFactor)

# Here the TOPO_DS Shape to be displayed is stored in the Lifting surface 'Shape' Attribute
c = display.DisplayShape(Wing.Shape, update=True)

# Mirror the wing:
W2 = act.mirror(Wing.Shape, plane='xz')
# from OCC.AIS import AIS_WireFrame
# display.Context.SetDisplayMode(AIS_WireFrame)
# display.Repaint()
display.DisplayShape(W2, update=True)

### File input/output

In [ ]:
# Write Output STEP file e.g. for viewing in Rhino:
act.export_STEPFile([Wing.Shape], 'wing.stp')

In [ ]:
# # OR use python 'pickle' (Though for files which will be sent, STEP is probably better)
# import pickle

# pickle.dump(Wing, open("wing.p", "wb"))

# del(Wing)
# display.EraseAll()

# LoadedWing = pickle.load(open("wing.p", "rb"))
# display.DisplayShape(LoadedWing.Shape, update=True, color='Blue')

## Tailplane, Transonic Airliner

In [ ]:
from OCC.gp import gp_Ax1, gp_Pnt, gp_Dir
from airconics.examples.tailplane_example_transonic_airliner import *

# Position of the apex of the fin
P = [36.98-0.49-0.02, 0.0, 2.395-0.141]

SegmentNo = 100
ChordFact = 1.01
ScaleFact = 21.93

Fin = liftingsurface.LiftingSurface(P, mySweepAngleFunctionFin,
                                    myDihedralFunctionFin,
                                    myTwistFunctionFin,
                                    myChordFunctionFin,
                                    myAirfoilFunctionFin,
                                    SegmentNo=SegmentNo,
                                    ChordFactor=ChordFact,
                                    ScaleFactor=ScaleFact)

#    Create the rotation axis centered at the apex point in the x direction
RotAxis = gp_Ax1(gp_Pnt(*P), gp_Dir(1, 0, 0))

# Having some problem with the fin loft: display some airfoils
# to figure out what's going on:
#    for section in Fin._Sections:
#        curve = section.Curve.GetObject()
#        curve.Scale(gp_Pnt(0., 0., 0.), ScaleFact)
#        display.DisplayShape(section.Curve, update=True)

Fin.Rotate(RotAxis, 90)
display.DisplayShape(Fin.Shape, update=True)

# Position of the apex of the tailplane
P = [43, 0.000, 1.633+0.02]

SegmentNo = 100
ChordFactor = 1.01
ScaleFactor = 17.3

TP = liftingsurface.LiftingSurface(P, mySweepAngleFunctionTP,
                                   myDihedralFunctionTP,
                                   myTwistFunctionTP,
                                   myChordFunctionTP,
                                   myAirfoilFunctionTP,
                                   SegmentNo=SegmentNo,
                                   ChordFactor=ChordFact,
                                   ScaleFactor=ScaleFact)

display.DisplayShape(TP.Shape, update=True)

TP2 = act.mirror(TP.Shape, plane='xz', copy=True)

# Note: TP2 is a TopoDS_Shape, not a wing and DisplayShape is called as:
display.DisplayShape(TP2, update=True)

## Fuselage Transonic Airliner

In [1]:
# Adding this here so I dont have to keep scrolling down: remove later
import airconics
import airconics.AirCONICStools as act
%gui qt4
# Initialise the display
from OCC.Display.SimpleGui import init_display
display, start_display, add_menu, add_function_to_menu = init_display()


Using PyQt4


In [5]:
Fus = airconics.fuselage_oml.Fuselage()

display.DisplayShape(Fus.PortCurve, update=True)

L = len(Fus._SectionPlanes)
face = Fus._SectionPlanes[L/2]
    
type(face)

(0.0, -2.9221499999999998, 0.0, 55.902000000000001, 0.0, 0.0)


OCC.TopoDS.TopoDS_Face

In [ ]:
# Testing stuff(remove later)

from OCC.Geom2d import Geom2d_BezierCurve
from OCC.TColgp import TColgp_Array1OfPnt2d
from OCC.Geom2d import Handle_Geom2d_Curve
from OCC.gp import gp_Pnt2d
from OCC.BRepBuilderAPI import BRepBuilderAPI_MakeEdge2d
import numpy as np 
from OCC.Bnd import Bnd_Box2d
from OCC.BRepBndLib import brepbndlib_Add

NoseLengthRatio = 0.182
TailLengthRatio = 0.293
kN = NoseLengthRatio / 0.182
tN = TailLengthRatio / 0.293
PlanPort = np.array([[0,                  0,     0],
                     [0*kN,              -0.1,   0],
                     [0.332*kN,          -0.395, 0],
                     [1.250*kN,          -0.810, 0],
                     [2.517*kN,          -1.074, 0],
                     [4*kN,              -1.15,  0],
                     [4*kN,              -1.15,  0],
                     # Parallel sided section here
                     [22-(22-15.55)*tN,  -1.15,   0],
                     [22-(22-15.55)*tN,  -1.15,   0],
                     [22-(22-16.428)*tN, -1.126,  0],
                     [22-(22-20.3362)*tN,-0.483,  0],
                     [22,                -0.0987, 0]])

array = PlanPort[:, 0:2]
N = array.shape[0]
pt_arr = TColgp_Array1OfPnt2d(0, N-1)
for i, pt in enumerate(array):
    pt_arr.SetValue(i, gp_Pnt2d(*pt))

PlanPortCurve = Geom2d_BezierCurve(pt_arr)
H_PlanPortCurve = Handle_Geom2d_Curve(PlanPortCurve)      # Get handle of curve      
PP_Edge = BRepBuilderAPI_MakeEdge2d(PlanPortCurve.GetHandle()).Edge()

display.DisplayShape(PlanPortCurve)

## Random things I tried out (Needs removing)

In [ ]:
# from OCC.gp import gp_Lin
# from OCC.ShapeConstruct import shapeconstruct

# Af2 = airconics.primitives.Airfoil([0.,0.,0.], ChordLength=3., Naca4Profile='2412')

# h = Af2.curve.GetHandle()
# EP = h.EndPoint()
# SP = h.StartPoint()
# closure = gp_Lin(EP, SP)

# shapeconstruct.JoinCurves(h.GetObject(), closure.)

# display.DisplayShape(Af2.Curve, update=True)

In [ ]:
# from OCC.gp import gp_Lin, gp_Ax1, gp_Dir
# from OCC.ShapeConstruct import shapeconstruct
# from OCC.BRepBuilderAPI import BRepBuilderAPI_MakeEdge

# Af2 = airconics.primitives.Airfoil([0.,0.,0.], ChordLength=3., Naca4Profile='2412')

# h = Af2.Curve.GetObject()
# EP = h.EndPoint()
# SP = h.StartPoint()
# closure = BRepBuilderAPI_MakeEdge(gp_Lin(gp_Ax1(EP, gp_Dir(SP.XYZ()))) ).Edge()

# display.DisplayShape(closure, update=True)
# display.DisplayShape(Af2.Curve, update=True)

In [ ]:
# # Note : This seems to make objects render in ipython notebook (only works for shapes? only works if %gui qt4 has npt)
import sys
import os

# Change this to path to pythonocc-contrib repository
pythonocc_contrib_dir = os.path.abspath('/home/pchambers/git/pythonocc-contrib')
sys.path.append(pythonocc_contrib_dir)

# Create the wing:
from airconics.examples.wing_example_transonic_airliner import *

# Position of the apex of the wing
P = (0,0,0)
NSeg = 10
ChordFactor = 1
ScaleFactor = 50
Wing = liftingsurface.LiftingSurface(P, mySweepAngleFunctionAirliner, 
    myDihedralFunctionAirliner, 
    myTwistFunctionAirliner, 
    myChordFunctionAirliner, 
    myAirfoilFunctionAirliner, SegmentNo=NSeg, ScaleFactor=ScaleFactor)

import WebServer.TornadoWeb
renderer = WebServer.TornadoWeb.TornadoWebRenderer()
renderer.DisplayShape(Wing.Shape)
renderer

In [ ]:
orient = Wing.Shape.Orientation()

In [ ]:
type(Af1.Curve)